# Alocação e otimização de portfólios

## Importando libs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

## Importando os dados

In [2]:
dataset = pd.read_csv('acoes.csv')
dataset

,Date,GOL,CVC,WEG,MGLU,TOTVS,IBOV
0,2015-01-02,14.99,11.164607,4.988705,2.058074,10.155536,47.259998
1,2015-01-05,14.85,11.017703,5.022718,2.096750,9.843493,46.320000
2,2015-01-06,15.21,10.870803,4.948212,2.069124,9.227921,46.580002
3,2015-01-07,14.55,10.775316,4.891521,2.138187,9.162673,48.150002
4,2015-01-08,14.27,10.393366,4.974128,2.121612,9.375432,48.509998
...,...,...,...,...,...,...,...
2477,2025-01-27,1.71,1.860000,53.310001,7.010000,31.209999,121.870003
2478,2025-01-28,1.71,1.770000,54.090000,6.760000,30.790001,121.029999
2479,2025-01-29,1.69,1.800000,54.430000,6.590000,31.150000,120.169998
2480,2025-01-30,1.70,1.950000,56.209999,7.270000,32.610001,123.949997


## Alocação aleatório de ativos

In [ ]:
def alocacao_ativos(dataset, dinheiro_total):
    dataset = dataset.copy()

    pesos = np.random.random(len(dataset.columns) - 1)
    pesos = pesos / pesos.sum()
    print(pesos, pesos.sum())

    colunas = dataset.columns[1:]

    for i in colunas:
        dataset[i] = ( dataset[i] / dataset[i][0])

    for i, acao in enumerate(dataset.columns[1:]):
        dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

    dataset['soma valor'] = dataset.iloc[:, 1:].sum(axis = 1)

    return dataset        

IndentationError: unindent does not match any outer indentation level (<string>, line 16)

In [30]:
alocacao_ativos(pd.read_csv('acoes.csv'), 5000)

[0.09731492 0.24023782 0.17954808 0.19998861 0.18996765 0.09294291] 1.0000000000000002


TypeError: can only concatenate str (not "float") to str